# Unsual Gates like $X(0)^{1.2n(1)}$ or sqrt(iSWAP)

The purpose of this notebook is to illustrate 2 new members
in the set of gates that Qubiter recognizes. The 2 gates are

1. an arbitrary matrix U2 in U(2), parametrized as 
$U2 = \exp(i[ \theta_0 + \theta_1\sigma_X + \theta_2\sigma_Y + \theta_3\sigma_Z])$ 
for real $\theta_j$, where $\sigma_X, \sigma_Y, \sigma_Z$ are the Pauli matrices.
2. a generalization of SWAP(0, 1) which I call SWAY(0, 1). SWAY includes SWAP, sqrt(SWAP), iSWAP, sqrt(iSWAP) etc. I discuss SWAY more precisely below.

The Qubiter simulator can now handle both of these gates with any number of controls of type T or F


## importance of $\theta_0$ when U2 has controls
Qubiter previously included ROTN, which is a U2 with $\theta_0=0$, which means ROTN is an 
element of SU(2), i.e., the Special 2-dim unitaries whose determinant is 1. Quantum mechanics does not
care about global phases, so why would we want a U2 with non-zero $\theta_0$?
Because when we attach one or more controls to a U2 with non-zero $\theta_0$,
the controls place conditions on that $\theta_0$, so the $\theta_0$ now has a physical 
significance and can no longer be dropped. For example, 
$$[e^{i\frac{\pi}{2} \sigma_Z(0)}]^{n(1)} = [i\sigma_Z(0)]^{n(1)}= i^{n(1)}\sigma_Z(0)^{n(1)}$$
is physically different from
$$[-ie^{i\frac{\pi}{2} \sigma_Z(0)}]^{n(1)}=\sigma_Z(0)^{n(1)}$$

## SWAY

Recall that the swap of two qubits 0, 1, call it SWAP(1, 0), is defined by 

$$ SWAP = diag(1, \sigma_X, 1) $$

> NOTE: SWAP is qbit symmetric, meaning that SWAP(0,1) = SWAP(1,0)

We define SWAY by

$$ SWAY = diag(1, U2, 1) $$

where U2 is the most general **symmetric** 2-dim unitary matrix
parametrized as

$$U2 = \exp(i[ \theta_0 + \theta_1\sigma_X + \theta_2\sigma_Y + \theta_3\sigma_Z])$$ 
for real $\theta_j$, with $$\theta_2=0$$


>NOTE: SWAY is qbit symmetric (SWAY(0,1)=SWAY(1,0)) iff U2 is a
symmetric matrix ($U2^T = U2$) iff $\theta_2=0$


Next, we will test these two new types of gates.

First change your working directory to the qubiter directory in your computer, and add its path to the path environment variable.

In [1]:
import os
import sys
print(os.getcwd())
os.chdir('../')
print(os.getcwd())
sys.path.insert(0,os.getcwd())

/home/jupyter/Notebooks/Quantum/qubiter/jupyter-notebooks
/home/jupyter/Notebooks/Quantum/qubiter


In [2]:
from SEO_writer import *
from SEO_simulator import *

In [3]:
num_bits = 5
emb = CktEmbedder(num_bits, num_bits)
trols = Controls(num_bits)
trols.bit_pos_to_kind = {3: True, 4: False}
trols.refresh_lists()
file_prefix = 'io_folder/sway_test'
g = np.pi/180
rads_list = ['#1', 20*g, 0, 40*g]

In [4]:
wr = SEO_writer(file_prefix, emb)
wr.write_X(3)
wr.write_U2(0, rads_list)
wr.write_controlled_one_bit_gate(0, trols, OneBitGates.u2, rads_list)

# swa_ stands for either swap or sway.

# without a rads_list, swa_ stands for swap
wr.write_bit_swa_(0, 1)
wr.write_controlled_bit_swa_(0, 1, trols)

# with a rads_list, swa_ stands for sway
wr.write_bit_swa_(0, 1, rads_list)
wr.write_controlled_bit_swa_(0, 1, trols, rads_list)

wr.close_files()

The above code wrote English and Picture files in the io_folder. We can ask wr to print them for us 

In [5]:
wr.print_eng_file()

SIGX	AT	3
U_2_	#1	20.0	0.0	40.0	AT	0
U_2_	#1	20.0	0.0	40.0	AT	0	IF	4F	3T
SWAP	1	0
SWAP	1	0	IF	4F	3T
SWAY	1	0	BY	#1	0.3490658503988659	0	0.6981317007977318
SWAY	1	0	BY	#1	0.3490658503988659	0	0.6981317007977318	IF	4F	3T



In [6]:
wr.print_pic_file()

|   X   |   |   |   
|   |   |   |   U   
O---@---+---+---U   
|   |   |   <--->   
O---@---+---<--->   
|   |   |   <<-->>  
O---@---+---<<-->>  



The following shows that class SEO_simulator recognizes the 2 new gates, with and without controls, and can multiply
through them

In [7]:
vman = PlaceholderManager(var_num_to_rads={1: np.pi/6})
sim = SEO_simulator(file_prefix, num_bits, verbose=False,
                    vars_manager=vman)
StateVec.describe_st_vec_dict(sim.cur_st_vec_dict)

*********branch= pure
total probability of state vector (=one if no measurements)= 1.0
dictionary with key=qubit, value=(Prob(0), Prob(1))
{0: (0.80004860313463511, 0.19995139686536489),
 1: (0.99997031678784598, 2.9683212154019145e-05),
 2: (1.0, 0.0),
 3: (0.0, 1.0),
 4: (1.0, 0.0)}
